In [26]:
import numpy as np
import pandas as pd
import sagemaker
import sys
import boto3
import io
import time
from time import gmtime,strftime,sleep
import datetime

In [27]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker/feature-store'

In [28]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
col_names = ['mpg','cylinders', 'displacement', 'horsepower', 'weight', 
           'acceleration', 'model_year', 'origin', 'car_name']
df = pd.read_csv(data_url, delimiter='\s+', header=None, names=col_names, na_values='?')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [29]:
df.sort_values(by=['car_name', 'model_year'])

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
96,13.0,8,360.0,175.0,3821.0,11.0,73,1,amc ambassador brougham
9,15.0,8,390.0,190.0,3850.0,8.5,70,1,amc ambassador dpl
66,17.0,8,304.0,150.0,3672.0,11.5,72,1,amc ambassador sst
257,19.4,6,232.0,90.0,3210.0,17.2,78,1,amc concord
315,24.3,4,151.0,90.0,3003.0,20.1,80,1,amc concord
...,...,...,...,...,...,...,...,...,...
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
197,29.0,4,90.0,70.0,1937.0,14.2,76,2,vw rabbit
309,41.5,4,98.0,76.0,2144.0,14.7,80,2,vw rabbit
325,44.3,4,90.0,48.0,2085.0,21.7,80,2,vw rabbit c (diesel)


In [30]:
df['car_name'] = df['car_name'].astype('string')

In [31]:
df['car_name']

0      chevrolet chevelle malibu
1              buick skylark 320
2             plymouth satellite
3                  amc rebel sst
4                    ford torino
                 ...            
393              ford mustang gl
394                    vw pickup
395                dodge rampage
396                  ford ranger
397                   chevy s-10
Name: car_name, Length: 398, dtype: string

In [32]:
d_df = {}
for yr in df['model_year'].unique():
    d_df[str(yr)] = df[df['model_year']==yr]
    d_df[str(yr)]['event_time'] = datetime.datetime(1900+yr, 1, 1, 8, 0, 0).timestamp() 
    
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
d_df['70'].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,28800.0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,28800.0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,28800.0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,28800.0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,28800.0


In [36]:
timestamp = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
feature_group_name = 'auto-mpg-%s' %timestamp
feature_group_name

'auto-mpg-2022-04-10-13-44-00'

In [37]:
from sagemaker.feature_store.feature_group import FeatureGroup
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session = sess)

In [39]:
record_identifier_feature_name = 'car_name'
event_time_feature_name = 'event_time'

feature_group.load_feature_definitions(data_frame=d_df['70'])

[FeatureDefinition(feature_name='mpg', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='cylinders', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='displacement', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='horsepower', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='weight', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='acceleration', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='model_year', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='origin', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='car_name', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractiona

In [40]:
description='This feature group tracks the vehicle information such as mpg, and horsepower between 1970 and 1982.'
len(description)

100

In [41]:
feature_group.create(
    s3_uri = f's3://{bucket}/{prefix}',
    enable_online_store = True,
    record_identifier_name = record_identifier_feature_name,
    event_time_feature_name = event_time_feature_name,
    description = description,
    role_arn = role)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:104877823522:feature-group/auto-mpg-2022-04-10-13-44-00',
 'ResponseMetadata': {'RequestId': '4c4b8e6e-e0c4-4cbb-8cbc-6a666b5b52eb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4c4b8e6e-e0c4-4cbb-8cbc-6a666b5b52eb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '105',
   'date': 'Sun, 10 Apr 2022 14:22:31 GMT'},
  'RetryAttempts': 0}}

In [42]:
def check_feature_group_status(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("waiting for Feature Group to be Created")
        time.sleep(15)
        status = feature_group.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group.name} successfully created")
    
check_feature_group_status(feature_group)

FeatureGroup auto-mpg-2022-04-10-13-44-00 successfully created


In [46]:
for yr, df_auto in d_df.items():
    print(yr)
    print(df_auto.shape)
    feature_group.ingest(data_frame=df_auto, max_workers=1, max_processes=1, wait=True)

70
(29, 10)
71
(28, 10)
72
(28, 10)
73
(40, 10)
74
(27, 10)
75
(30, 10)
76
(34, 10)
77
(28, 10)
78
(36, 10)
79
(29, 10)
80
(29, 10)
81
(29, 10)
82
(31, 10)


In [47]:
car_name = 'amc concord'
featurestore_runtime = sess.boto_session.client(service_name='sagemaker-featurestore-runtime',
                                               region_name=region)
sample_record = featurestore_runtime.get_record(FeatureGroupName=feature_group_name,
                                               RecordIdentifierValueAsString=car_name)

In [48]:
sample_record

{'ResponseMetadata': {'RequestId': 'b068d3ae-c599-4321-8f14-0ea979c16c82',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b068d3ae-c599-4321-8f14-0ea979c16c82',
   'content-type': 'application/json',
   'content-length': '527',
   'date': 'Sun, 10 Apr 2022 22:48:39 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'mpg', 'ValueAsString': '24.3'},
  {'FeatureName': 'cylinders', 'ValueAsString': '4'},
  {'FeatureName': 'displacement', 'ValueAsString': '151.0'},
  {'FeatureName': 'horsepower', 'ValueAsString': '90.0'},
  {'FeatureName': 'weight', 'ValueAsString': '3003.0'},
  {'FeatureName': 'acceleration', 'ValueAsString': '20.1'},
  {'FeatureName': 'model_year', 'ValueAsString': '80'},
  {'FeatureName': 'origin', 'ValueAsString': '1'},
  {'FeatureName': 'car_name', 'ValueAsString': 'amc concord'},
  {'FeatureName': 'event_time', 'ValueAsString': '315561600.0'}]}

In [52]:
query = feature_group.athena_query()
table_name = query.table_name

query_string = ('SELECT * FROM "%s"' %table_name)
print('Running ' + query_string)

query.run(query_string = query_string,
         output_location=f's3://{bucket}/{prefix}/query_results/')

query.wait()
dataset = query.as_dataframe()

Running SELECT * FROM "auto-mpg-2022-04-10-13-44-00-1649600551"


In [53]:
dataset.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted
0,26.0,4,116.0,75.0,2246.0,14.0,74,2,fiat 124 tc,126259200.0,2022-04-10 22:39:07.148,2022-04-10 22:34:18.000,False
1,27.0,4,112.0,88.0,2640.0,18.6,82,1,chevrolet cavalier wagon,378720000.0,2022-04-10 22:39:12.494,2022-04-10 22:34:24.000,False
2,30.0,4,146.0,67.0,3250.0,21.8,80,2,mercedes-benz 240d,315561600.0,2022-04-10 22:39:09.287,2022-04-10 22:34:23.000,False
3,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,28800.0,2022-04-10 22:39:07.116,2022-04-10 22:34:14.000,False
4,31.0,4,119.0,82.0,2720.0,19.4,82,1,chevy s-10,378720000.0,2022-04-10 22:39:07.784,2022-04-10 22:34:25.000,False


In [54]:
dataset.shape

(398, 13)

In [55]:
query_string_2 = '''
select * from "%s" where model_year < 79 ''' %table_name
print('Running ' + query_string_2)

query.run(query_string = query_string_2,
         output_location = f's3://{bucket}/{prefix}/query_results/')
query.wait()
dataset_2 = query.as_dataframe()

Running 
select * from "auto-mpg-2022-04-10-13-44-00-1649600551" where model_year < 79 


In [56]:
dataset_2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted
0,31.0,4,79.0,67.0,1950.0,19.0,74,3,datsun b210,126259200.0,2022-04-10 22:39:08.359,2022-04-10 22:34:18.000,False
1,22.0,4,122.0,86.0,2395.0,16.0,72,1,ford pinto (sw),63100800.0,2022-04-10 22:39:10.212,2022-04-10 22:34:16.000,False
2,16.0,6,225.0,105.0,3439.0,15.5,71,1,plymouth satellite custom,31564800.0,2022-04-10 22:39:07.016,2022-04-10 22:34:15.000,False
3,19.0,6,250.0,88.0,3302.0,15.5,71,1,ford torino 500,31564800.0,2022-04-10 22:39:07.016,2022-04-10 22:34:15.000,False
4,23.8,4,151.0,85.0,2855.0,17.6,78,1,oldsmobile starfire sx,252489600.0,2022-04-10 22:39:13.960,2022-04-10 22:34:21.000,False


In [58]:
dataset_2.shape

(280, 13)

In [61]:
query_string3 = '''
select * from (
select *, row_number() over (partition by car_name order by event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_number from "%s" where event_time < %.f) where row_number = 1 and NOT is_deleted 
''' % (table_name, datetime.datetime(1979, 1, 1, 8, 0, 0).timestamp())

print('Running '+ query_string3)

query.run(
    query_string=query_string3,
    output_location = f's3://{bucket}/{prefix}/query_results')
query.wait()
dataset_3 = query.as_dataframe()

Running 
select * from (
select *, row_number() over (partition by car_name order by event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_number from "auto-mpg-2022-04-10-13-44-00-1649600551" where event_time < 284025600) where row_number = 1 and NOT is_deleted 



In [62]:
dataset_3.shape

(212, 14)

In [63]:
dataset_2[dataset_2['car_name']=='amc gremlin']

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted
168,19.0,6,232.0,100.0,2634.0,13.0,71,1,amc gremlin,31564800.0,2022-04-10 22:39:14.822,2022-04-10 22:34:15.000,False
175,21.0,6,199.0,90.0,2648.0,15.0,70,1,amc gremlin,28800.0,2022-04-10 22:39:14.835,2022-04-10 22:34:15.000,False
190,20.0,6,232.0,100.0,2914.0,16.0,75,1,amc gremlin,157795200.0,2022-04-10 22:39:14.821,2022-04-10 22:34:19.000,False
217,18.0,6,232.0,100.0,2789.0,15.0,73,1,amc gremlin,94723200.0,2022-04-10 22:39:14.828,2022-04-10 22:34:17.000,False


In [64]:
dataset_3[dataset_3['car_name']=='amc gremlin']

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted,row_number
4,20.0,6,232.0,100.0,2914.0,16.0,75,1,amc gremlin,157795200.0,2022-04-10 22:39:14.821,2022-04-10 22:34:19.000,False,1


In [65]:
car_name = 'amc gremlin'
featurestore_runtime =  sess.boto_session.client(service_name='sagemaker-featurestore-runtime', 
                                                 region_name=region)
amc_gremlin = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=car_name)

amc_gremlin['Record']

[{'FeatureName': 'mpg', 'ValueAsString': '20.0'},
 {'FeatureName': 'cylinders', 'ValueAsString': '6'},
 {'FeatureName': 'displacement', 'ValueAsString': '232.0'},
 {'FeatureName': 'horsepower', 'ValueAsString': '100.0'},
 {'FeatureName': 'weight', 'ValueAsString': '2914.0'},
 {'FeatureName': 'acceleration', 'ValueAsString': '16.0'},
 {'FeatureName': 'model_year', 'ValueAsString': '75'},
 {'FeatureName': 'origin', 'ValueAsString': '1'},
 {'FeatureName': 'car_name', 'ValueAsString': 'amc gremlin'},
 {'FeatureName': 'event_time', 'ValueAsString': '157795200.0'}]

In [66]:
car_names = ['amc gremlin', 'amc concord', 'dodge colt']
feature_names = ['cylinders', 'displacement', 'horsepower']
sample_batch_records=featurestore_runtime.batch_get_record(
   Identifiers=[
     {
       'FeatureGroupName': feature_group_name,
       'RecordIdentifiersValueAsString': car_names,
       'FeatureNames': feature_names
     },
   ]
)
sample_batch_records['Records'][0]['Record']

[{'FeatureName': 'cylinders', 'ValueAsString': '4'},
 {'FeatureName': 'displacement', 'ValueAsString': '151.0'},
 {'FeatureName': 'horsepower', 'ValueAsString': '90.0'}]

In [ ]:
# feature_group.delete()